# Operaciones esenciales en un **RDD**

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext('local[*]', 'Mas sobre RDD')

In [ ]:
sc

## Crear input directamente

### Ya hemos visto que podemos transformar cualquier colleción en un RDD mediante `sc.parallelize`. Cuando queremos crear un conjunto de números con un espaciamiento constante podemos emplear `sc.range`; este método funciona igual que la versión de **python**, con la diferencia que esto genera un archivo distribuido **RDD** que contará con particiones y estas se enviarán a distintas computadoras o hilos de nuestro equipo para este caso.

In [ ]:
numeros = sc.range(100000)
numeros.take(8)

In [ ]:
numeros.getNumPartitions()

## Podemos aplicar una función de **python** a cada elemento del **RDD**

In [ ]:
cuadrados = numeros.map(lambda x: x**2)
cuadrados.take(15)

In [ ]:
tuplas_cuadrados = cuadrados.map(lambda x: ( x, len(str(x))) )
tuplas_cuadrados.take(10)

## Podemos escoger un subconjunto del **RDD** creando filtros.

### El método `filter` de un **RDD** toma como argumento una función $f$ y retorna un **RDD** que corresponde a aquellos elementos $\{x_i\}$ para los cuales $f(x_i)$ retorna **True**. De esta manera, la función $f$ debe tomar un elemento del **RDD** como argumento y regresar **True** o **False**. 

In [ ]:
# Seleccionar aquellos elementos cuyo número de dígitos sea un número par
digitos_pares = tuplas_cuadrados.filter( lambda x: x[1] % 2 == 0  )
digitos_pares.take(10)

In [ ]:
# Reordenar para que el número de dígitos corresponda a la llave (key)
llave_valor = digitos_pares.map(lambda x: (x[1], x[0]))
llave_valor.take(10)

## Otra operación fundamental es la de agrupar utilizando las llaves.

### En este caso se crean grupos para las llaves: 2, 4, 6, etc.

In [ ]:
grupos = llave_valor.groupByKey()
grupos.take(4)

### Por default los elementos no se encuantran ordenados. Podemos ordenar mediante la llave con: 

In [ ]:
grupos = grupos.sortByKey()
grupos.take(1)

### Podemos emplear el siguiente truco si deseamos ver el contenido de los grupos (esto no es necesario para los cálculos pero es una ayuda visual para que el programador sepa que hace lo correcto)

In [ ]:
grupos = grupos.map( lambda x: (x[0], list(x[1]) ) )
grupos.take(1)

## Reducción

### Otra operación fundamental es la reducción, la cual podemos llevar a cabo una vez el agrupamiento ha terminado

In [ ]:
# La reducción en este caso es el valor promedio
promedios = grupos.map( lambda x: (x[0], sum(x[1])/len(x[1])) )
promedios.collect()

In [ ]:
sc.stop()